# Classifying Celestial Objects
Based on information from the dataset neo_v2 from https://www.kaggle.com/datasets/sameepvani/nasa-nearest-earth-objects, this neural network classifies whether asteroids may pose a danger to earth based on a variety of features. 

### Imports

In [1]:
import tensorflow as tf 
from tensorflow import keras 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import requests
import io
import os
import tempfile

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

relative_velocity_flag = False
miss_distance_flag = False

### Data preprocessing
The neo_v2 csv file is imported from the REHS github repo. Then the program splits the dataset 80/20 into training data and testing data respectively

In [2]:
csv_link = 'https://raw.githubusercontent.com/sdsc-hpc-students/REHS2022/main/Final-Project/ExtraKGs/celesital-bodies/neo_v2.csv'
csv_file = requests.get(csv_link).content

In [3]:
dataset = pd.read_csv(io.StringIO(csv_file.decode('utf-8')))

In [4]:
dataset.head()

,id,name,est_diameter_min,est_diameter_max,relative_velocity,miss_distance,orbiting_body,sentry_object,absolute_magnitude,hazardous
0,2162635,162635 (2000 SS164),1.198271,2.679415,13569.249224,5.483974e+07,Earth,False,16.73,False
1,2277475,277475 (2005 WK4),0.265800,0.594347,73588.726663,6.143813e+07,Earth,False,20.00,True
2,2512244,512244 (2015 YE18),0.722030,1.614507,114258.692129,4.979872e+07,Earth,False,17.83,False
3,3596030,(2012 BV13),0.096506,0.215794,24764.303138,2.543497e+07,Earth,False,22.20,False
4,3667127,(2014 GE35),0.255009,0.570217,42737.733765,4.627557e+07,Earth,False,20.09,True


convert boolean values to floats

In [5]:
print(dataset.dtypes.tolist())
print(type(dataset.dtypes.tolist()[0]))

change_to_float = {column:float for column, dtype in zip(dataset.columns.tolist(), dataset.dtypes.tolist()) if dtype == np.bool_}
dataset = dataset.astype(change_to_float)
dataset.head()

[dtype('int64'), dtype('O'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('O'), dtype('bool'), dtype('float64'), dtype('bool')]
<class 'numpy.dtype[int64]'>


,id,name,est_diameter_min,est_diameter_max,relative_velocity,miss_distance,orbiting_body,sentry_object,absolute_magnitude,hazardous
0,2162635,162635 (2000 SS164),1.198271,2.679415,13569.249224,5.483974e+07,Earth,0.0,16.73,0.0
1,2277475,277475 (2005 WK4),0.265800,0.594347,73588.726663,6.143813e+07,Earth,0.0,20.00,1.0
2,2512244,512244 (2015 YE18),0.722030,1.614507,114258.692129,4.979872e+07,Earth,0.0,17.83,0.0
3,3596030,(2012 BV13),0.096506,0.215794,24764.303138,2.543497e+07,Earth,0.0,22.20,0.0
4,3667127,(2014 GE35),0.255009,0.570217,42737.733765,4.627557e+07,Earth,0.0,20.09,1.0


get rid of orbiting body and sentry_object

In [6]:
try:
    dataset.drop('sentry_object', axis=1, inplace=True)
except (AttributeError, KeyError):
    print("already removed")

try:
    dataset.drop('orbiting_body', axis=1, inplace=True)
except (AttributeError, KeyError):
    print("already removed")

In [7]:
dataset.head()

,id,name,est_diameter_min,est_diameter_max,relative_velocity,miss_distance,absolute_magnitude,hazardous
0,2162635,162635 (2000 SS164),1.198271,2.679415,13569.249224,5.483974e+07,16.73,0.0
1,2277475,277475 (2005 WK4),0.265800,0.594347,73588.726663,6.143813e+07,20.00,1.0
2,2512244,512244 (2015 YE18),0.722030,1.614507,114258.692129,4.979872e+07,17.83,0.0
3,3596030,(2012 BV13),0.096506,0.215794,24764.303138,2.543497e+07,22.20,0.0
4,3667127,(2014 GE35),0.255009,0.570217,42737.733765,4.627557e+07,20.09,1.0


what percent of our data is positive?

In [8]:
neg, pos = np.bincount(dataset['hazardous'])
total = neg + pos
print(f"Percent positive is {(pos / neg) * 100} %")

Percent positive is 10.781013707985755 %


In [9]:
dataset_raw = dataset # to look at later

In [10]:
def scale_down(dataset=None, column=None):
    initial_range = np.amax(dataset[column]) - np.amin(dataset[column])
    eps = 0.001
    dataset[column] = np.log(dataset[column] + eps)
    end_range = np.amax(dataset[column]) - np.amin(dataset[column])

    return initial_range, end_range

In [11]:
# try to reduce the range of the miss distance
if not relative_velocity_flag:
    initial_range, end_range = scale_down(column='miss_distance', dataset=dataset)
    relative_velocity_flag = True

else:
    print("scaling already complete")

In [12]:
# try to reduce the range of the relative velocity
if not miss_distance_flag:
    initial_range, end_range = scale_down(column='relative_velocity', dataset=dataset)
    miss_distance_flag = True
else:
    print("scaling already complete")

In [13]:
scaler = StandardScaler()

In [14]:
train_data, test_data = train_test_split(dataset.drop(['id', 'name'], axis=1), test_size=0.2)
train_data, validation_data = train_test_split(train_data, test_size=0.2)

train_labels, test_labels, validation_labels = (np.array(train_data.pop('hazardous')),
                                                np.array(test_data.pop('hazardous')), 
                                                np.array(validation_data.pop('hazardous')))
bool_train_labels = train_labels != 0
train_features, test_features, validation_features = (np.clip(scaler.fit_transform(np.array(train_data)), 0, 60),
                                                      np.clip(scaler.fit_transform(np.array(test_data)), 0, 60), 
                                                      np.clip(scaler.fit_transform(np.array(validation_data)), 0, 60))

In [15]:
model_metrics = [keras.metrics.BinaryAccuracy(),
                       keras.metrics.Precision(),
                       keras.metrics.Recall(),
                       keras.metrics.TruePositives(),
                       keras.metrics.TrueNegatives(),
                       keras.metrics.FalseNegatives(),
                       keras.metrics.FalsePositives(),
                       keras.metrics.AUC(name='auc'),
                       keras.metrics.AUC(name='prc', curve='PR')
                       ]

In [16]:
def make_model(metrics=model_metrics, output_bias=None):
    if output_bias:
        output_bias = tf.keras.initializers.Constant(output_bias)
    
    model = tf.keras.Sequential([
        keras.layers.Dense(5, activation='relu', input_shape=(train_features.shape[-1],), 
                            kernel_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-4),
                            bias_regularizer=keras.regularizers.L2(1e-4)),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(5, activation='linear', activity_regularizer=keras.regularizers.L1(0.0002)),
        keras.layers.Dense(4, activation='relu', input_shape=(train_features.shape[-1],),
                            kernel_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-4),
                            bias_regularizer=keras.regularizers.L2(1e-4),),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(3, activation='linear', activity_regularizer=keras.regularizers.L1(0.0002)),
        keras.layers.Dense(3, activation='relu', input_shape=(train_features.shape[-1],),
                            kernel_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-4),
                            bias_regularizer=keras.regularizers.L2(1e-4)),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(2, activation='linear', activity_regularizer=keras.regularizers.L1(0.0002)),
        keras.layers.Dense(1, activation='sigmoid', bias_initializer=output_bias)
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=metrics
    )
    
    return model

In [17]:
EPOCHS = 100
BATCH_SIZE = 2048

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_prc',
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True
)

In [18]:
model = make_model()

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 5)                 30        
                                                                 
 dropout (Dropout)           (None, 5)                 0         
                                                                 
 dense_1 (Dense)             (None, 5)                 30        
                                                                 
 dense_2 (Dense)             (None, 4)                 24        
                                                                 
 dropout_1 (Dropout)         (None, 4)                 0         
                                                                 
 dense_3 (Dense)             (None, 3)                 15        
                                                                 
 dense_4 (Dense)             (None, 3)                 1

getting the initial bias

In [20]:
# unbiased
results = model.evaluate(train_features, train_labels, batch_size=BATCH_SIZE, verbose=1)
print(f'Loss: {results[0]}')

29/29 [==============================] - 2s 4ms/step - loss: 0.5920 - binary_accuracy: 0.8332 - precision: 0.2133 - recall: 0.2739 - true_positives: 1530.0000 - true_negatives: 46905.0000 - false_negatives: 4055.0000 - false_positives: 5644.0000 - auc: 0.7300 - prc: 0.1907
Loss: 0.5919867157936096


initial_bias = ln(pos/neg)
where positive is the number of features classified positive, and neg is the number of features classified as negative

In [21]:
initial_bias = np.log([pos/neg])
initial_bias

array([-2.22738359])

In [22]:
model = make_model(output_bias=initial_bias)

In [23]:
model.predict(train_features[:10])

1/1 [==============================] - 0s 199ms/step


array([[0.09378146],
       [0.09349285],
       [0.09731824],
       [0.08969834],
       [0.09327094],
       [0.09327585],
       [0.0907452 ],
       [0.09731824],
       [0.09392492],
       [0.09723588]], dtype=float32)

In [24]:
# biased
results = model.evaluate(train_features, train_labels, batch_size=BATCH_SIZE, verbose=1)
print(f'Loss: {results[0]}')

29/29 [==============================] - 2s 3ms/step - loss: 0.3178 - binary_accuracy: 0.8685 - precision: 0.2133 - recall: 0.1370 - true_positives: 1530.0000 - true_negatives: 99454.0000 - false_negatives: 9640.0000 - false_positives: 5644.0000 - auc: 0.5760 - prc: 0.1457
Loss: 0.31775376200675964


save a benchmark

In [25]:
initial_weights = os.path.join(tempfile.mkdtemp(), 'initial_weights')
model.save_weights(initial_weights)

In [26]:
model = make_model()
model.load_weights(initial_weights)

baseline = model.fit(train_features, train_labels, batch_size=BATCH_SIZE, epochs=EPOCHS, callbacks=[early_stopping], validation_data=(validation_features, validation_labels))

Epoch 1/100
29/29 [==============================] - 6s 63ms/step - loss: 0.3163 - binary_accuracy: 0.8803 - precision: 0.2132 - recall: 0.0913 - true_positives: 1530.0000 - true_negatives: 152002.0000 - false_negatives: 15225.0000 - false_positives: 5645.0000 - auc: 0.5473 - prc: 0.1305 - val_loss: 0.3223 - val_binary_accuracy: 0.9004 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_true_positives: 0.0000e+00 - val_true_negatives: 13087.0000 - val_false_negatives: 1447.0000 - val_false_positives: 0.0000e+00 - val_auc: 0.7465 - val_prc: 0.1864
Epoch 2/100
29/29 [==============================] - 0s 8ms/step - loss: 0.3149 - binary_accuracy: 0.9039 - precision: 0.3333 - recall: 1.7905e-04 - true_positives: 1.0000 - true_negatives: 52547.0000 - false_negatives: 5584.0000 - false_positives: 2.0000 - auc: 0.5413 - prc: 0.1056 - val_loss: 0.3192 - val_binary_accuracy: 0.9004 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_true_positives: 0.0000e+00 - val_true_negatives:

class weights

In [27]:
weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)

class_weights = {0: weight_for_0, 1: weight_for_1}

print(f'Non-Hazardous Weight: {weight_for_0}\nHazardous Weight: {weight_for_1}')

Non-Hazardous Weight: 0.5539050685399288
Hazardous Weight: 5.137782805429865


In [28]:
tf.keras.backend.clear_session()

In [29]:
weighted_model = make_model()
weighted_model.load_weights(initial_weights)

In [30]:
# weighted_history = weighted_model.fit(train_features, 
#                                     train_labels, 
#                                     batch_size=BATCH_SIZE, 
#                                     epochs=EPOCHS, 
#                                     callbacks=[early_stopping], 
#                                     validation_data=(validation_features, validation_labels), 
#                                     class_weight=class_weights)

try training it with just the weights

In [31]:
weighted_model.fit(train_features, 
                    train_labels, 
                    batch_size=BATCH_SIZE, 
                    epochs=EPOCHS, 
                    #callbacks=[early_stopping], 
                    validation_data=(validation_features, validation_labels), 
                    class_weight=class_weights)

Epoch 1/100
29/29 [==============================] - 6s 72ms/step - loss: 1.1959 - binary_accuracy: 0.9032 - precision: 1.0000 - recall: 1.4221e-04 - true_positives: 1.0000 - true_negatives: 65636.0000 - false_negatives: 7031.0000 - false_positives: 0.0000e+00 - auc: 0.6343 - prc: 0.1626 - val_loss: 0.3247 - val_binary_accuracy: 0.9004 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_true_positives: 0.0000e+00 - val_true_negatives: 13087.0000 - val_false_negatives: 1447.0000 - val_false_positives: 0.0000e+00 - val_auc: 0.5000 - val_prc: 0.0996
Epoch 2/100
29/29 [==============================] - 0s 8ms/step - loss: 1.1634 - binary_accuracy: 0.9039 - precision: 0.0000e+00 - recall: 0.0000e+00 - true_positives: 0.0000e+00 - true_negatives: 52546.0000 - false_negatives: 5585.0000 - false_positives: 3.0000 - auc: 0.5459 - prc: 0.1086 - val_loss: 0.3233 - val_binary_accuracy: 0.9004 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_true_positives: 0.0000e+00 - val_true_ne

In [32]:
weighted_model.evaluate(test_features, test_labels, batch_size=BATCH_SIZE, verbose=1)

9/9 [==============================] - 0s 4ms/step - loss: 0.3786 - binary_accuracy: 0.7744 - precision: 0.2862 - recall: 0.8485 - true_positives: 1534.0000 - true_negatives: 12535.0000 - false_negatives: 274.0000 - false_positives: 3825.0000 - auc: 0.8555 - prc: 0.2747


[0.3786129057407379,
 0.774383544921875,
 0.2862474322319031,
 0.8484513163566589,
 1534.0,
 12535.0,
 274.0,
 3825.0,
 0.8555399179458618,
 0.2747349739074707]

In [33]:
results = weighted_model.predict(test_features)
results[results<0.5]=0
results[results>0.5]=1

568/568 [==============================] - 1s 2ms/step


In [34]:
results.shape
results = results.reshape(18168,)
comparison = np.stack([results, test_labels, test_])

NameError: name 'test_' is not defined

In [ ]:
comparison_tuples = list(zip(comparison[0], comparison[1]))

In [ ]:
def check_correctness(comparison):
    for predicted_label, actual_label in comparison:
        if predicted_label == actual_label:
            print(True)
            continue
        print(False)

In [ ]:
check_correctness(comparison_tuples)

Oversampling

In [ ]:
# pos_features, neg_features = train_features[bool_train_labels], train_features[~bool_train_labels]

# pos_labels, neg_labels = train_labels[bool_train_labels], train_labels[~bool_train_labels]

In [ ]:
# BUFFER_SIZE = 100000

# def make_ds(features, labels):
#     ds = tf.data.Dataset.from_tensor_slices((features, labels))
#     ds = ds.shuffle(BUFFER_SIZE).repeat()
#     return ds

In [ ]:
# pos_ds, neg_ds = make_ds(pos_features, pos_labels), make_ds(neg_features, neg_labels)

In [ ]:
# resampled_ds = tf.data.Dataset.sample_from_datasets([pos_ds, neg_ds], weights=[0.5, 0.5])
# resampled_ds = resampled_ds.batch(BATCH_SIZE).prefetch(2)

# resampled_steps_per_epoch = np.ceil(2.0*neg/BATCH_SIZE)

train with oversampling

In [ ]:
# resampled_model = make_model()
# resampled_model.load_weights(initial_weights)

# # reset bias to 0, the set is now balanced
# output_layer = resampled_model.layers[-1]
# output_layer.bias.assign([0])

# validation_data = tf.data.Dataset.from_tensor_slices((validation_data, validation_labels)).cache()
# validation_data = validation_data.batch(BATCH_SIZE).prefetch(2)

In [ ]:
# resampled_history = resampled_model.fit(resampled_ds,
#                                         epochs=EPOCHS,
#                                         steps_per_epoch=resampled_steps_per_epoch,
#                                         callbacks=[early_stopping],
#                                         validation_data=validation_data)

make the resample model

In [ ]:
# resampled_model.fit(resampled_ds,
#                     epochs=EPOCHS,
#                     steps_per_epoch=resampled_steps_per_epoch,
#                     #callbacks=[early_stopping],
#                     validation_data=validation_data,
#                     class_weight={0:0.1, 1:0.1})

In [ ]:
# resampled_model.evaluate(test_features, test_labels, batch_size=BATCH_SIZE, verbose=1)

In [ ]:
tf.keras.backend.clear_session()